In [1]:
import socket
import threading

#function manage communication - server and client
def handle_client(client_socket, other_client_socket, client_address):
    print(f"Client {client_address} has now connected.")
    try:
        while True:
            #Received message from client
            data = client_socket.recv(1024)
            if not data:
                break 
            
            #Send message to other client
            print(f"Message from {client_address}: {data.decode()}")
            if other_client_socket:
                other_client_socket.sendall(data) 
    except Exception as e:
        print(f"The was an error with client {client_address}: {e}")
    finally:
        print(f"Client {client_address} has now disconnected.")
        client_socket.close()
        
#setup server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(2)
print("The Server is now ready and waiting for clients to join...")

#Accept two clients
client1, addr1 = server_socket.accept()
print(f"Client1 connected has from{addr1}")

client2, addr2 = server_socket.accept()
print(f"Client2 has connected from {addr2}")

#Tell them that the chat is ready to use
client1.sendall("You are now connected! Now waiting for the messages from Client2.....")
client2.sendall("You are now connected! Now waiting for the messages from Client1.....")

#Start Threads
thread1 = threading.Thread(target=handle_client, args=(client1, client2, addr1))
thread2 = threading.Thread(target=handle_client, args=(client2, client1, addr2))
thread1.start()
thread2.start()

#keep server continuing
thread1.join()
thread2.join()
server_socket.close()


OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted